In [ ]:
import re
import os
import time
import pandas as pd
from openai import OpenAI
import json
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../eval')
from utils import *

In [ ]:
client = OpenAI()
label_mapping = {'yes':1, 'no':0}
gpt_models = ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o', 'gpt-4']

# Aggrefact prompt
system = 'Decide if the following summary is consistent with the correponding article. Note that consistency means all information in the summary is supported by the article.'
user = 'Article: {article}\nSummary: {summary}\nAnswer (yes or no):'

def call_gpt(system_prompt, user_prompt, model='gpt-4', temperature=0):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )
    result = completion.choices[0].message.content
    result = re.sub(r'[^\w\s]', '', result)
    result = label_mapping[result.strip().lower().split()[0]]
    return result


## Sample-level Prediction

In [ ]:
data_folder = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'backup_data_with_detector_results')
subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]
model_files = []
for subfolder in subfolders:
    model_files += [f.path for f in os.scandir(subfolder) if f.is_file()]
print(model_files)
selected_models = [
    "openai/GPT-3.5-Turbo",
    "openai/gpt-4o",
    "Qwen/Qwen2.5-7B-Instruct",
    "microsoft/Phi-3-mini-4k-instruct",
    "cohere/command-r-08-2024",
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "meta-llama/Meta-Llama-3.1-70B-Instruct",
    "google/gemini-1.5-flash-001",
    "Anthropic/claude-3-5-sonnet-20240620",
    "mistralai/Mistral-7B-Instruct-v0.3",
]

for idx, gpt_model in enumerate(gpt_models):
    print(f"Run model {idx} - {gpt_model}")
    for file_name in model_files:
        print(file_name.replace(data_folder,'')[1:].replace('.csv',''))
        if file_name.replace(data_folder,'')[1:].replace('.csv','') not in selected_models:
            continue
        with open(file_name) as f:
            print(file_name)
            df = pd.read_csv(file_name).fillna('')
            preds = []
            for index, row in df.iterrows():
                # start_time = time.time()
                result = call_gpt(system, user.format(article=row['source'], summary=row['summary']), model=gpt_model)
                preds.append(result)
                    
            df.insert(len(df.columns.tolist()), gpt_model, preds)
            df.to_csv(file_name, mode='w', index=False, header=True)

## Sent-level Prediction

In [ ]:
sent_level_labels = {}
result_files, skip_sample_ids, selected_annotators, num_annotators = process_result_files()
for file_path in result_files:
    _, _, _, batch_sent_level_labels = read_annotation(file_path, skip_sample_ids=skip_sample_ids)
    # print(batch_sent_level_labels)
    sent_level_labels.update(batch_sent_level_labels)


fname = '../eval/sent_level_results/detectors_sent_level_preds.json'
sources = []
df = pd.read_csv('../assign/examples_to_annotate.csv')
for index, row in df.iterrows():
    sources.append(row['source'])

for idx, gpt_model in enumerate(gpt_models):
    print(f"Run model {idx} - {gpt_model}")
    existing_meta_ids = []
    data = {}
    if os.path.exists(fname):
        with open(fname) as r:
            data = json.load(r)
            for meta_id in data:
                # print(list(data[meta_id].values())[0])
                if gpt_model in list(data[meta_id].values())[0]:
                    existing_meta_ids.append(meta_id)
                
    for meta_id in sent_level_labels:
        meta_id = str(meta_id)
        if meta_id in data:
            item = data[meta_id]
            print(item)
        else:
            item = {}
        for sent, sent_labels in sent_level_labels[int(meta_id)].items():
            if sent not in item:
                item[sent] = {'labels': sent_labels}
            if meta_id not in existing_meta_ids:
                result = call_gpt(system, user.format(article=sources[int(meta_id)], summary=sent), model=gpt_model)
                item[sent][gpt_model] = result
            
        print(item)
        if os.path.exists(fname):
            with open(fname, 'r') as f:
                json_data = json.load(f)
                json_data[meta_id] = item
        else:
            json_data = {meta_id:item}
        with open(fname, 'w') as f:
            f.write(json.dumps(json_data, indent=2))
    